In [1]:
import pandas as pd
import json

# Define the file path and necessary columns
file_path = r'C:\Users\xxxxx\Desktop\interconnection_legislative_documents\01_phrase_selection\phrasing001\gpt_dataset_6.json' 
necessary_columns = ['men_id', 'men_text', 'phrase', 'men_href', 'men_href_title', 'doc_id']

# Initialize an empty list to store the filtered data
filtered_data = []

# Read the JSON file line by line
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # Parse the JSON object
        data = json.loads(line)
        # Extract the necessary columns
        filtered_data.append({col: data[col] for col in necessary_columns})

# Create a DataFrame from the filtered data
df = pd.DataFrame(filtered_data)

In [2]:
df.head()

,men_id,men_text,phrase,men_href,men_href_title,doc_id
0,420349,art. 135,Minuta deciziei nr. 8 din 17 mai 2021 Admite r...,https://legislatie.just.ro/Public/DetaliiDocum...,CODUL DE PROCEDURĂ CIVILĂ din 1 iulie 2010 (**...,19797
1,420350,art. 136 din Codul de procedură civilă,Minuta deciziei nr. 8 din 17 mai 2021 Admite r...,https://legislatie.just.ro/Public/DetaliiDocum...,CODUL DE PROCEDURĂ CIVILĂ din 1 iulie 2010 (**...,19797
2,420351,art. 54 din Legea nr. 304/2004,Minuta deciziei nr. 8 din 17 mai 2021 Admite r...,https://legislatie.just.ro/Public/DetaliiDocum...,LEGE nr. 304 din 28 iunie 2004 (**republicată**),19797
3,420352,art. 31 alin. (2) din Legea nr. 304/2004,Completul de judecată competent să soluționeze...,https://legislatie.just.ro/Public/DetaliiDocum...,LEGE nr. 304 din 28 iunie 2004 (**republicată**),19797
4,420353,art. 136 alin. (3) din Codul de procedură civilă,Completul de judecată competent să soluționeze...,https://legislatie.just.ro/Public/DetaliiDocum...,CODUL DE PROCEDURĂ CIVILĂ din 1 iulie 2010 (**...,19797


In [3]:
df.shape

(67072, 6)

In [4]:
import pickle
import re
import spacy
nlp = spacy.load("ro_core_news_lg")

In [5]:
def purify_and_vectorize(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    # Remove phone numbers
    # text = re.sub(r'\b\d{10}\b|\b\d{3}[-.\s]\d{3}[-.\s]\d{4}\b', '', text)
    # Process text with spaCy
    doc = nlp(text)
    # Filter out stop words and punctuations, lemmatize, and get vectors
    vectors = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    # Convert lemmatized words to their vectors
    # vectors = [nlp(word).vector for word in vectors if word in nlp.vocab]
    return vectors

In [6]:
# Save the DataFrame into a pickle file
with open('preprocessed_dataframe_6.pkl', 'wb') as f:
    pickle.dump(pd.DataFrame({
        'men_id': df['men_id'],
        'men_text': df['men_text'].apply(purify_and_vectorize),
        'phrase': df['phrase'].apply(purify_and_vectorize),
        'men_href_title': df['men_href_title'].apply(purify_and_vectorize),
        'men_href': df['men_href'],
        'doc_id': df['doc_id']
    }), f)